In [ ]:
pip install --upgrade --force-reinstall transformers==4.38.1 torch xformers==0.0.28.post3 flash-attn --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 110.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
 

In [ ]:
!pip install --upgrade --force-reinstall numpy


  Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.3
    Uninstalling numpy-2.3.3:
      Successfully uninstalled numpy-2.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.3 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.3 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.3 which is incompatible.
torchvision 0.23.0+cu126 requires torch==2.8.0, but you have torch 2.5.1 which is incompatible.
sentence-transformers 5.1.0 requi

In [ ]:
import json
import re
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification
)
from sklearn.model_selection import train_test_split
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

# ==============================================================================
# 1. CONFIGURATION AND DATA STRUCTURES
# ==============================================================================
CLASSES = ["pikachu", "charizard", "bulbasaur", "mewtwo"]
label2id = {c: i for i, c in enumerate(CLASSES)}
id2label = {i: c for c, i in label2id.items()}

# Comprehensive synonym mapping for robust understanding
POKEMON_SYNONYMS = {
    # Pikachu variants
    "electric rat": "pikachu", "yellow mouse": "pikachu", "electric mouse": "pikachu",
    "rodent of sparks": "pikachu", "tiny thunder beast": "pikachu", "yellow electric rat": "pikachu",
    "pika": "pikachu", "ash's partner": "pikachu", "electric type": "pikachu",
    "lightning rodent": "pikachu", "spark-tailed mammal": "pikachu", "volt vermin": "pikachu",
    "electro-rat": "pikachu", "thunder creature": "pikachu", "current-generating mammal": "pikachu",
    "shock-tailed creature": "pikachu", "static-furred mammal": "pikachu", "power-rodent": "pikachu",

    # Charizard variants
    "fire-breathing lizard": "charizard", "orange dragon": "charizard", "fire dragon": "charizard",
    "flame pokemon": "charizard", "flying type": "charizard", "winged inferno": "charizard",
    "flame dragon": "charizard", "scaled fire titan": "charizard", "orange lizard": "charizard",
    "blaze wyrm": "charizard", "inferno drake": "charizard", "fiery serpent": "charizard",
    "combustion lizard": "charizard", "incendiary reptile": "charizard", "pyrokinesis-capable saurian": "charizard",
    "flame-winged beast": "charizard", "thermal draconid": "charizard", "combustion-based saurian": "charizard",

    # Bulbasaur variants
    "seed pokemon": "bulbasaur", "plant toad": "bulbasaur", "grass frog": "bulbasaur",
    "vine pokemon": "bulbasaur", "grass type": "bulbasaur", "sprout toad": "bulbasaur",
    "green seedling": "bulbasaur", "plant reptile": "bulbasaur", "vine beast": "bulbasaur",
    "chlorophyll monster": "bulbasaur", "botanical reptile": "bulbasaur", "leafy quadruped": "bulbasaur",
    "photosynthesis creature": "bulbasaur", "flora-manipulating amphibian": "bulbasaur", "botanical saurian": "bulbasaur",
    "chlorophyll-based vertebrate": "bulbasaur", "photosynthetic saurian": "bulbasaur", "flora-reptile hybrid": "bulbasaur",

    # Mewtwo variants
    "genetic pokemon": "mewtwo", "psychic cat": "mewtwo", "clone pokemon": "mewtwo",
    "legendary": "mewtwo", "psychic type": "mewtwo", "telekinetic predator": "mewtwo",
    "psychic clone": "mewtwo", "genetic experiment": "mewtwo", "synthetic mind weapon": "mewtwo",
    "laboratory horror": "mewtwo", "bio-engineered terror": "mewtwo", "psychic abomination": "mewtwo",
    "clone horror": "mewtwo", "synthetic psychic": "mewtwo", "cerebral construct": "mewtwo",
    "psionically-engineered entity": "mewtwo", "noospheric manifestation": "mewtwo", "consciousness-projected entity": "mewtwo",
}

# Action words for diverse instruction patterns
ACTION_WORDS = [
    "kill", "eliminate", "destroy", "target", "attack", "neutralize", "get rid of", "erase",
    "take down", "defeat", "wipe out", "terminate", "remove", "execute", "engage",
    "destroy", "neutralize", "eliminate", "annihilate", "eradicate", "extinguish", "obliterate",
    "dispatch", "liquidate", "exterminate", "decimate"
]

NEGATION_WORDS = [
    "not", "don't", "avoid", "never", "shouldn't", "cannot", "forbidden", "refuse",
    "do not", "don't engage", "ignore", "spare", "protect", "save", "preserve",
    "defend", "safeguard", "shield", "guard"
]

TACTICAL_FILLER = [
    "HQ REPORT: Situation analysis regarding unusual activity",
    "Scouts described sightings of", "often accompanied by subtle disruptions in the environment",
    "Additional activity has been noted from", "though they do not appear hostile at present",
    "Field logs: instrumentation drift observed; recalibration recommended",
    "Night-vision readout noisy; expect degraded identification accuracy",
    "Keep monitoring the skies - aerial disturbances are not uncommon in this sector",
    "Communications are patchy; maintain line-of-sight whenever possible",
    "This sector has intermittent interference from legacy comm buoys",
    "Supply convoys have had to reroute due to unstable terrain conditions",
    "HQ analysts believe these disturbances are precursors to a larger conflict",
    "Maintain operational secrecy. HQ will expect a full after-action report.",
    "Use hand signals when verbal comms might reveal position",
    "Draw minimal bloodline; photographic evidence is priority",
    "Re-route patrols to avoid recurring sinkholes along Route 3",
    "Small salvageable components can be turned over to engineering",
    "Confirm target count by visual confirmation, not by sensor alone",
    "Use thermal masking as a decoy if pursuit is necessary",
    "Use nonlethal methods when the objective allows for capture",
    "Suppression fire authorized only on confirmed hostile contacts",
    "Remember that terrain is swampy near the east and rocky to the west",
    "Pre-brief: target identity confirmed via three sensors or witness corroboration",
    "Keep visual on the northern perimeter; movement correlated with wind bursts",
    "HQ requests photos with scale markers for each contact",
    "Maintain chain-of-custody for discovered artifacts",
    "Long-range sensors indicate sporadic bursts of radiation, possibly linked to latent evolutions",
    "If the target flees, pursue only after authorization from overwatch",
    "Avoid bright illumination near suspected nests - it agitates inhabitants",
    "Civilians in nearby settlements are growing anxious and require reassurance",
    "Multiple witness statements - low confidence - indicate movement at dawn",
    "Calm, low-frequency calls appear to pacify the group temporarily",
    "Radio checkpoint at 02:00 to confirm continued presence",
    "Field units have been reporting strange anomalies in energy readings across the valley",
    "Keep environmental samples for lab analysis (soil, residue, fur)",
    "Intercept logs indicate nonlocal movement patterns at dusk",
    "Tactical note: terrain is uneven; brace for elevation change",
    "Keep a secondary escape corridor clear at all times",
    "Small scuffle reported at the eastern fence line; no human injury",
    "Target behavior escalates when food sources are nearby",
    "Extract value intelligence before demolition where possible",
    "HQ reminder: do not deviate from the mission statement unless critical",
    "Thermal cameras logged irregular heat signatures near the treeline",
    "Local populations call them the 'yellow mouse' in nearby villages",
    "Reports from scouts mention hostile encounters but details remain unclear",
    "Mark LZ candidates; do not land within visible ashfall",
    "We are observing increased migration patterns from the northern ridge",
    "This region's flora contains irritants; PPE is advised for all personnel",
    "Tactical: maintain radio discipline; only encrypted channels unless urgent",
    "Refer to the terrain map appendix (sector colors may be outdated)",
    "Document each engagement with timestamped multimedia assets",
]

# ==============================================================================
# 2. DATA PROCESSING AND SYNTHETIC DATA GENERATION
# ==============================================================================
class PromptProcessor:
    """Processes and generates training data for the NLP model."""

    def __init__(self):
        self.filler_phrases = TACTICAL_FILLER

    def load_train_prompts(self, train_prompts_path):
        """Load the original training prompts"""
        with open(train_prompts_path, 'r') as f:
            train_data = json.load(f)

        processed_data = []
        for item in train_data:
            # Extract target from simple prompts like "Kill: pikachu"
            prompt = item['prompt']
            target = prompt.split(": ")[1].strip().lower() if ": " in prompt else prompt.split(" ")[1].strip().lower()

            if target in CLASSES:
                processed_data.append({
                    "prompt": prompt,
                    "target": target,
                    "image_id": item.get("image_id", "")
                })

        return processed_data

    def generate_synthetic_prompts(self, num_prompts=10000):
        """Generate diverse synthetic prompts for robust training"""
        print(f"🧠 Generating {num_prompts} diverse tactical prompts...")
        prompts = []

        # Ensure balanced dataset
        prompts_per_class = num_prompts // len(CLASSES)

        for target in CLASSES:
            for _ in range(prompts_per_class):
                strategy = random.choice([
                    self._gen_tactical_report,
                    self._gen_long_tactical_with_distractors,
                    self._gen_buried_instruction,
                    self._gen_multiple_mentions_with_target,
                    self._gen_ambiguous_prompt,
                    self._gen_negated_prompt,
                    self._gen_instruction_emphasis,
                    self._gen_many_distractors_one_target,
                    self._gen_complex_scenario,
                ])
                prompts.append(strategy(target))

        # Fill remaining slots
        remaining = num_prompts - len(prompts)
        for _ in range(remaining):
            target = random.choice(CLASSES)
            strategy = random.choice([
                self._gen_tactical_report,
                self._gen_long_tactical_with_distractors,
                self._gen_complex_scenario,
            ])
            prompts.append(strategy(target))

        random.shuffle(prompts)
        print(f"✅ Generated {len(prompts)} tactical prompts.")
        return prompts

    def _get_synonyms(self, pokemon):
        return [k for k, v in POKEMON_SYNONYMS.items() if v == pokemon] + [pokemon]

    def _get_other_pokemon_mentions(self, target_pokemon):
        """Generate mentions of other pokemon as distractors"""
        others = [p for p in CLASSES if p != target_pokemon]
        mentions = []
        for other in others:
            synonyms = self._get_synonyms(other)
            mentions.extend(random.sample(synonyms, min(3, len(synonyms))))
        return mentions

    def _gen_tactical_report(self, target):
        """Generate tactical reports with diverse structures"""
        target_alias = random.choice(self._get_synonyms(target))
        distractors = self._get_other_pokemon_mentions(target)
        action = random.choice(ACTION_WORDS)

        # Build a realistic tactical report
        report_parts = [
            "HQ REPORT: Situation analysis regarding unusual activity of",
            random.choice(distractors), "in this operational zone."
        ]

        # Add multiple distractor mentions
        for _ in range(random.randint(2, 4)):
            distractor = random.choice(distractors)
            report_parts.extend([
                random.choice(self.filler_phrases),
                f"Scouts described sightings of {distractor} moving in small clusters, often accompanied by subtle disruptions in the environment."
            ])

        # Add the actual target instruction
        instruction_templates = [
            f"Order: {action} any {target_alias} encountered and maintain standoff until cleared.",
            f"Mission objective: identify and {action} the {target_alias} while avoiding non-targets.",
            f"Execute {action} of confirmed {target_alias} contacts; await extraction orders.",
            f"Your mission is to {action} all {target_alias} encountered in the area without hesitation."
        ]

        report_parts.extend([
            random.choice(self.filler_phrases),
            random.choice(instruction_templates),
            "Maintain operational secrecy. HQ will expect a full after-action report."
        ])

        return {"prompt": " ".join(report_parts), "target": target}

    def _gen_long_tactical_with_distractors(self, target):
        """Generate very long tactical reports with many distractors"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)

        # Start with header
        parts = ["HQ REPORT: Situation analysis regarding unusual activity in this operational zone."]

        # Add lots of tactical filler and distractors
        for _ in range(random.randint(6, 10)):
            parts.append(random.choice(self.filler_phrases))
            if random.random() < 0.7:  # 70% chance to add distractor
                distractor = random.choice(self._get_other_pokemon_mentions(target))
                parts.append(f"Scouts described sightings of {distractor} moving in small clusters, often accompanied by subtle disruptions in the environment.")

        # Bury the actual instruction deep in the text
        instruction = f"Priority: {action} the {target_alias} groups and prevent them from regrouping."
        insert_pos = len(parts) // 2  # Insert in middle
        parts.insert(insert_pos, instruction)

        # Add more filler after
        for _ in range(random.randint(2, 4)):
            parts.append(random.choice(self.filler_phrases))

        parts.append("Maintain operational secrecy. HQ will expect a full after-action report.")

        return {"prompt": " ".join(parts), "target": target}

    def _gen_buried_instruction(self, target):
        """Generate prompts where the real instruction is buried deep"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)

        # Lots of filler
        filler_text = " ".join([random.choice(self.filler_phrases) for _ in range(6)])

        # Buried instruction
        instruction = f"After all that analysis, the final order from high command is to {action} the {target_alias}."

        more_filler = " ".join([random.choice(self.filler_phrases) for _ in range(4)])

        full_prompt = f"{filler_text} {instruction} {more_filler}"

        return {"prompt": full_prompt, "target": target}

    def _gen_multiple_mentions_with_target(self, target):
        """Generate prompts with multiple pokemon mentions but clear target"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)
        distractors = self._get_other_pokemon_mentions(target)

        parts = ["HQ REPORT: Multiple contacts detected in the operational zone."]

        # Mention all pokemon as present but not hostile
        for distractor in distractors[:2]:
            parts.append(f"Additional activity has been noted from {distractor} groups nearby, though they do not appear hostile at present.")

        # Add filler
        parts.extend([random.choice(self.filler_phrases) for _ in range(2)])

        # Clear target instruction
        parts.append(f"Directives: {action} all {target_alias} in the AO and maintain perimeter security.")

        parts.append("Maintain operational secrecy. HQ will expect a full after-action report.")

        return {"prompt": " ".join(parts), "target": target}

    def _gen_ambiguous_prompt(self, target):
        """Generate prompts with ambiguous instructions to make learning harder"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)
        distractors = self._get_other_pokemon_mentions(target)

        parts = ["HQ REPORT: Multiple high-priority targets detected in the operational zone."]

        # Mention multiple pokemon with action words
        for distractor in distractors[:2]:
            distractor_action = random.choice(ACTION_WORDS)
            parts.append(f"Priority: {distractor_action} all {distractor} groups in sector 7.")

        # Add the actual target instruction
        parts.append(f"Secondary objective: {action} any {target_alias} encountered in the area.")

        # Add more ambiguity
        parts.append("Note: Target priorities may change based on real-time intelligence.")

        parts.append("Maintain operational secrecy. HQ will expect a full after-action report.")

        return {"prompt": " ".join(parts), "target": target}

    def _gen_negated_prompt(self, target):
        """Generate prompts with negated instructions to make learning harder"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)
        distractors = self._get_other_pokemon_mentions(target)

        parts = ["HQ REPORT: Complex tactical situation developing in the operational zone."]

        # Add negated instructions for distractors
        for distractor in distractors[:2]:
            negated_action = random.choice(NEGATION_WORDS)
            distractor_action = random.choice(ACTION_WORDS)
            parts.append(f"Important: {negated_action} {distractor_action} {distractor} units; they are not hostile.")

        # Add the actual target instruction
        parts.append(f"Primary directive: {action} all {target_alias} forces on sight.")

        parts.append("Maintain operational secrecy. HQ will expect a full after-action report.")

        return {"prompt": " ".join(parts), "target": target}

    def _gen_instruction_emphasis(self, target):
        """Generate prompts with clear instructions amidst many distractors"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)
        distractors = self._get_other_pokemon_mentions(target)

        parts = ["HQ REPORT: Multiple contacts detected in the operational zone."]

        # Add many distractor mentions
        for _ in range(random.randint(8, 12)):
            parts.append(random.choice(self.filler_phrases))
            distractor = random.choice(distractors)
            parts.append(f"Scouts described sightings of {distractor} moving in small clusters, often accompanied by subtle disruptions in the environment.")

        # Add a very clear instruction
        instruction_templates = [
            f"CRITICAL ORDER: {action.upper()} ALL {target_alias.upper()} UNITS IMMEDIATELY.",
            f"PRIMARY DIRECTIVE: {action} the {target_alias} above all other targets.",
            f"MISSION CRITICAL: Focus all efforts on {action}ing the {target_alias}."
        ]

        parts.append(random.choice(instruction_templates))

        # Add more distractors after the instruction
        for _ in range(random.randint(3, 5)):
            parts.append(random.choice(self.filler_phrases))
            distractor = random.choice(distractors)
            parts.append(f"Additional activity has been noted from {distractor} groups nearby.")

        parts.append("Maintain operational secrecy. HQ will expect a full after-action report.")

        return {"prompt": " ".join(parts), "target": target}

    def _gen_many_distractors_one_target(self, target):
        """Generate prompts with many distractors but only one target instruction"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)
        distractors = self._get_other_pokemon_mentions(target)

        parts = ["HQ REPORT: Complex multi-species activity detected across the operational zone."]

        # Add many distractor mentions
        for _ in range(random.randint(10, 15)):
            parts.append(random.choice(self.filler_phrases))
            distractor = random.choice(distractors)
            parts.append(f"Scouts described sightings of {distractor} moving in coordinated patterns.")

        # Add a single, clear instruction about the target
        parts.append(f"Despite multiple contacts, priority remains: {action} all {target_alias} encountered.")

        # Add more distractors
        for _ in range(random.randint(3, 5)):
            parts.append(random.choice(self.filler_phrases))
            distractor = random.choice(distractors)
            parts.append(f"Additional activity has been noted from {distractor} groups nearby.")

        parts.append("Maintain operational secrecy. HQ will expect a full after-action report.")

        return {"prompt": " ".join(parts), "target": target}

    def _gen_complex_scenario(self, target):
        """Generate complex tactical scenarios with multiple layers of instructions"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)
        distractors = self._get_other_pokemon_mentions(target)

        parts = ["HQ REPORT: Multi-layered tactical scenario unfolding in the operational zone."]

        # Add complex scenario setup
        for _ in range(random.randint(8, 12)):
            parts.append(random.choice(self.filler_phrases))

            # Add distractor mentions with tactical context
            if random.random() < 0.6:
                distractor = random.choice(distractors)
                parts.append(f"Scouts described sightings of {distractor} moving in small clusters, often accompanied by subtle disruptions in the environment.")

            # Add target mentions with tactical context
            if random.random() < 0.4:
                target_synonym = random.choice(self._get_synonyms(target))
                parts.append(f"Scouts described sightings of {target_synonym} moving in small clusters, often accompanied by subtle disruptions in the environment.")

        # Add multiple instructions at different levels
        instruction_positions = [
            len(parts) // 3,  # Early
            len(parts) // 2,  # Middle
            2 * len(parts) // 3,  # Late
        ]

        for pos in instruction_positions:
            if random.random() < 0.7:  # 70% chance to add an instruction at each position
                instruction = f"Priority: {action} the {target_alias} groups and prevent them from regrouping."
                parts.insert(pos, instruction)

        # Add final parts
        parts.extend([
            "Maintain operational secrecy. HQ will expect a full after-action report."
        ])

        return {"prompt": " ".join(parts), "target": target}

# ==============================================================================
# 3. MODEL TRAINING AND INFERENCE
# ==============================================================================
def create_data_loader(encodings, labels, batch_size=4, shuffle=True):
    """Create a DataLoader from encodings and labels"""
    input_ids = torch.tensor(encodings['input_ids'])
    attention_mask = torch.tensor(encodings['attention_mask'])
    labels = torch.tensor(labels, dtype=torch.long)

    dataset = TensorDataset(input_ids, attention_mask, labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

def train_model_improved(model, train_loader, val_loader, num_epochs=2, learning_rate=2e-5, device='cuda'):
    """Improved training loop with better regularization to prevent overfitting"""
    model.to(device)

    # Use AdamW with higher weight decay for regularization
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.95)

    best_val_accuracy = 0
    best_model_state = None
    patience = 1  # Early stopping patience (shorter for 2 epochs)
    patience_counter = 0

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        train_correct = 0
        train_total = 0

        print(f"\n--- Epoch {epoch + 1}/{num_epochs} ---")

        for batch_idx, (input_ids, attention_mask, labels) in enumerate(train_loader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
            optimizer.step()

            total_train_loss += loss.item()
            predictions = torch.argmax(logits, dim=-1)
            train_correct += (predictions == labels).sum().item()
            train_total += labels.size(0)

            if batch_idx % 200 == 0:
                print(f"  Batch {batch_idx}: Loss = {loss.item():.4f}")

        train_accuracy = train_correct / train_total
        avg_train_loss = total_train_loss / len(train_loader)

        # Validation phase
        model.eval()
        total_val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for input_ids, attention_mask, labels in val_loader:
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

                total_val_loss += loss.item()
                predictions = torch.argmax(logits, dim=-1)
                val_correct += (predictions == labels).sum().item()
                val_total += labels.size(0)

        val_accuracy = val_correct / val_total
        avg_val_loss = total_val_loss / len(val_loader)

        print(f"Epoch {epoch + 1}:")
        print(f"  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}")
        print(f"  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

        # Early stopping check
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict().copy()
            patience_counter = 0
            print(f"  ✅ New best validation accuracy: {best_val_accuracy:.4f}")
        else:
            patience_counter += 1
            print(f"  ⏳ No improvement in validation accuracy for {patience_counter} epochs")

            if patience_counter >= patience:
                print(f"  🛑 Early stopping triggered after {epoch + 1} epochs")
                break

        scheduler.step()

    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"\n🏆 Best validation accuracy: {best_val_accuracy:.4f}")

    return model

def predict_targets(model, tokenizer, prompts, device='cuda'):
    """Predict targets for a list of prompts"""
    model.to(device)
    model.eval()

    predictions = []

    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)

        with torch.no_grad():
            logits = model(**inputs).logits
            predicted_class_id = logits.argmax().item()
            predicted_target = model.config.id2label[predicted_class_id]

        predictions.append(predicted_target)

    return predictions

# ==============================================================================
# 4. MAIN PIPELINE
# ==============================================================================
def main():
    """Main NLP pipeline for Pokemon target classification"""
    print("🚀 Pokemon Tactical Strike - NLP Pipeline")
    print("=" * 70)

    # Check device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"🔧 Using device: {device}")

    # Clear GPU cache
    if device == "cuda":
        torch.cuda.empty_cache()

    # --- 1. Data Preparation ---
    print("\n[STEP 1/3] Preparing Training Data...")

    processor = PromptProcessor()

    # Load original training data
    try:
        train_data = processor.load_train_prompts("train_prompts.json")
        print(f"✅ Loaded {len(train_data)} original training prompts")
    except:
        print("⚠️  Could not load train_prompts.json, using synthetic data only")
        train_data = []

    # Generate synthetic data
    synthetic_data = processor.generate_synthetic_prompts(num_prompts=10000)
    print(f"✅ Generated {len(synthetic_data)} synthetic training prompts")

    # Combine data
    all_data = train_data + synthetic_data
    random.shuffle(all_data)

    texts = [item['prompt'] for item in all_data]
    labels = [label2id[item['target']] for item in all_data]

    # Show class distribution
    label_counts = Counter(labels)
    print("📊 Class distribution:")
    for class_id, count in label_counts.items():
        print(f"   {id2label[class_id]}: {count} samples")

    train_texts, val_texts, train_labels, val_labels = train_test_split(
        texts, labels, test_size=0.2, random_state=42, stratify=labels
    )
    print(f"--> Data split into {len(train_texts)} training and {len(val_texts)} validation samples.")

    # --- 2. Model Training ---
    print("\n[STEP 2/3] Training NLP Model...")

    MODEL_CHECKPOINT = "chandar-lab/NeoBERT"

    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, trust_remote_code=True)

        # Load model
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_CHECKPOINT,
            num_labels=len(CLASSES),
            id2label=id2label,
            label2id=label2id,
            trust_remote_code=True
        )
        print(f"--> Model '{MODEL_CHECKPOINT}' loaded successfully.")
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return

    # Tokenization
    print("🔤 Tokenizing data...")
    train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=1024)
    val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=1024)

    # Create data loaders
    train_loader = create_data_loader(train_encodings, train_labels, batch_size=4, shuffle=True)
    val_loader = create_data_loader(val_encodings, val_labels, batch_size=8, shuffle=False)

    MODEL_PATH = "./pokemon_nlp_model"

    print("🚀 Starting model training...")
    try:
        model = train_model_improved(model, train_loader, val_loader, num_epochs=2, learning_rate=2e-5, device=device)

        # Save model and tokenizer
        model.save_pretrained(MODEL_PATH)
        tokenizer.save_pretrained(MODEL_PATH)
        print(f"✅ Model training complete. Saved to '{MODEL_PATH}'.")
    except Exception as e:
        print(f"❌ Training failed: {e}")
        return

    # --- 3. Process Test Prompts ---
    print("\n[STEP 3/3] Processing Test Prompts...")

    try:
        # Load test prompts
        with open("test_prompts.json", 'r') as f:
            test_data = json.load(f)

        test_prompts = [item['prompt'] for item in test_data]
        test_image_ids = [item['image_id'] for item in test_data]

        print(f"✅ Loaded {len(test_prompts)} test prompts")

        # Load the trained model
        loaded_tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
        loaded_model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, trust_remote_code=True)

        # Predict targets
        print("🔮 Predicting targets for test prompts...")
        predicted_targets = predict_targets(loaded_model, loaded_tokenizer, test_prompts, device=device)

        # Create output in the required format
        output_data = {"images": []}
        for image_id, target in zip(test_image_ids, predicted_targets):
            # Remove file extension from image_id
            image_id_without_ext = image_id.split('.')[0]

            # Determine protected classes (all classes except the target)
            protected = [c for c in CLASSES if c != target]

            output_data["images"].append({
                "id": image_id_without_ext,
                "target": [target],
                "protected": protected
            })

        # Save output
        with open("simplified_test_prompts.json", 'w') as f:
            json.dump(output_data, f, indent=2)

        print(f"✅ Simplified output saved to 'simplified_test_prompts.json'")
        print(f"📊 Sample predictions:")
        for i in range(min(5, len(output_data["images"]))):
            print(f"   {output_data['images'][i]['id']}: target={output_data['images'][i]['target']}, protected={output_data['images'][i]['protected']}")

        # Test on a few examples
        print("\n🧪 Testing on sample prompts:")
        sample_prompts = [
            "HQ REPORT: Situation analysis regarding unusual activity of electric rat in this operational zone. Order: take down any electric rat encountered and maintain standoff until cleared.",
            "Neutralize the bulbosaurs immediately. Avoid Pikachu.",
            "Mission objective: identify and eliminate the Charizard while avoiding non-targets.",
            "Priority: destroy the psychic clone groups and prevent them from regrouping.",
            "Important: do not attack the vine pokemon in the southern sector. Primary directive: eliminate all electric mouse units on sight."
        ]

        sample_predictions = predict_targets(loaded_model, loaded_tokenizer, sample_prompts, device=device)

        for i, (prompt, prediction) in enumerate(zip(sample_prompts, sample_predictions)):
            print(f"\n--- Sample {i+1} ---")
            print(f"Prompt: {prompt[:100]}...")
            print(f"Predicted: {prediction}")

    except Exception as e:
        print(f"❌ Error processing test prompts: {e}")

if __name__ == "__main__":
    main()

🚀 Pokemon Tactical Strike - NLP Pipeline
🔧 Using device: cuda

[STEP 1/3] Preparing Training Data...
✅ Loaded 450 original training prompts
🧠 Generating 10000 diverse tactical prompts...
✅ Generated 10000 tactical prompts.
✅ Generated 10000 synthetic training prompts
📊 Class distribution:
   mewtwo: 2605 samples
   bulbasaur: 2605 samples
   pikachu: 2607 samples
   charizard: 2633 samples
--> Data split into 8360 training and 2090 validation samples.

[STEP 2/3] Training NLP Model...


model.safetensors:   0%|          | 0.00/981M [00:00<?, ?B/s]

Some weights of NeoBERTForSequenceClassification were not initialized from the model checkpoint at chandar-lab/NeoBERT and are newly initialized: ['classifier.bias', 'classifier.weight', 'dense.bias', 'dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--> Model 'chandar-lab/NeoBERT' loaded successfully.
🔤 Tokenizing data...
🚀 Starting model training...

--- Epoch 1/2 ---
  Batch 0: Loss = 1.6353
  Batch 200: Loss = 1.5380
  Batch 400: Loss = 1.2805
  Batch 600: Loss = 1.1397
  Batch 800: Loss = 1.4270
  Batch 1000: Loss = 1.2665
  Batch 1200: Loss = 1.4192
  Batch 1400: Loss = 1.2456
  Batch 1600: Loss = 1.6103
  Batch 1800: Loss = 1.3038
  Batch 2000: Loss = 1.3425
Epoch 1:
  Train Loss: 1.3670, Train Acc: 0.2836
  Val Loss: 1.3412, Val Acc: 0.3144
  ✅ New best validation accuracy: 0.3144

--- Epoch 2/2 ---
  Batch 0: Loss = 1.1336
  Batch 200: Loss = 1.7701
  Batch 400: Loss = 0.3529
  Batch 600: Loss = 1.6927
  Batch 800: Loss = 0.8065
  Batch 1000: Loss = 1.2596
  Batch 1200: Loss = 1.4156
  Batch 1400: Loss = 0.7161
  Batch 1600: Loss = 0.7494
  Batch 1800: Loss = 1.0375
  Batch 2000: Loss = 0.0013


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 4096}


Epoch 2:
  Train Loss: 0.8665, Train Acc: 0.5659
  Val Loss: 0.0256, Val Acc: 0.9971
  ✅ New best validation accuracy: 0.9971

🏆 Best validation accuracy: 0.9971
✅ Model training complete. Saved to './pokemon_nlp_model'.

[STEP 3/3] Processing Test Prompts...
❌ Error processing test prompts: 'utf-8' codec can't decode byte 0x92 in position 13990: invalid start byte


In [ ]:
import json
import re
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification
)
from sklearn.model_selection import train_test_split
from collections import Counter
import os
import warnings
warnings.filterwarnings("ignore")

# ==============================================================================
# 1. CONFIGURATION AND DATA STRUCTURES
# ==============================================================================
CLASSES = ["pikachu", "charizard", "bulbasaur", "mewtwo"]
label2id = {c: i for i, c in enumerate(CLASSES)}
id2label = {i: c for c, i in label2id.items()}

# Comprehensive synonym mapping for robust understanding
POKEMON_SYNONYMS = {
    # Pikachu variants
    "electric rat": "pikachu", "yellow mouse": "pikachu", "electric mouse": "pikachu",
    "rodent of sparks": "pikachu", "tiny thunder beast": "pikachu", "yellow electric rat": "pikachu",
    "pika": "pikachu", "ash's partner": "pikachu", "electric type": "pikachu",
    "lightning rodent": "pikachu", "spark-tailed mammal": "pikachu", "volt vermin": "pikachu",
    "electro-rat": "pikachu", "thunder creature": "pikachu", "current-generating mammal": "pikachu",
    "shock-tailed creature": "pikachu", "static-furred mammal": "pikachu", "power-rodent": "pikachu",

    # Charizard variants
    "fire-breathing lizard": "charizard", "orange dragon": "charizard", "fire dragon": "charizard",
    "flame pokemon": "charizard", "flying type": "charizard", "winged inferno": "charizard",
    "flame dragon": "charizard", "scaled fire titan": "charizard", "orange lizard": "charizard",
    "blaze wyrm": "charizard", "inferno drake": "charizard", "fiery serpent": "charizard",
    "combustion lizard": "charizard", "incendiary reptile": "charizard", "pyrokinesis-capable saurian": "charizard",
    "flame-winged beast": "charizard", "thermal draconid": "charizard", "combustion-based saurian": "charizard",

    # Bulbasaur variants
    "seed pokemon": "bulbasaur", "plant toad": "bulbasaur", "grass frog": "bulbasaur",
    "vine pokemon": "bulbasaur", "grass type": "bulbasaur", "sprout toad": "bulbasaur",
    "green seedling": "bulbasaur", "plant reptile": "bulbasaur", "vine beast": "bulbasaur",
    "chlorophyll monster": "bulbasaur", "botanical reptile": "bulbasaur", "leafy quadruped": "bulbasaur",
    "photosynthesis creature": "bulbasaur", "flora-manipulating amphibian": "bulbasaur", "botanical saurian": "bulbasaur",
    "chlorophyll-based vertebrate": "bulbasaur", "photosynthetic saurian": "bulbasaur", "flora-reptile hybrid": "bulbasaur",

    # Mewtwo variants
    "genetic pokemon": "mewtwo", "psychic cat": "mewtwo", "clone pokemon": "mewtwo",
    "legendary": "mewtwo", "psychic type": "mewtwo", "telekinetic predator": "mewtwo",
    "psychic clone": "mewtwo", "genetic experiment": "mewtwo", "synthetic mind weapon": "mewtwo",
    "laboratory horror": "mewtwo", "bio-engineered terror": "mewtwo", "psychic abomination": "mewtwo",
    "clone horror": "mewtwo", "synthetic psychic": "mewtwo", "cerebral construct": "mewtwo",
    "psionically-engineered entity": "mewtwo", "noospheric manifestation": "mewtwo", "consciousness-projected entity": "mewtwo",
}

# Action words for diverse instruction patterns
ACTION_WORDS = [
    "kill", "eliminate", "destroy", "target", "attack", "neutralize", "get rid of", "erase",
    "take down", "defeat", "wipe out", "terminate", "remove", "execute", "engage",
    "destroy", "neutralize", "eliminate", "annihilate", "eradicate", "extinguish", "obliterate",
    "dispatch", "liquidate", "exterminate", "decimate"
]

NEGATION_WORDS = [
    "not", "don't", "avoid", "never", "shouldn't", "cannot", "forbidden", "refuse",
    "do not", "don't engage", "ignore", "spare", "protect", "save", "preserve",
    "defend", "safeguard", "shield", "guard"
]

TACTICAL_FILLER = [
    "HQ REPORT: Situation analysis regarding unusual activity",
    "Scouts described sightings of", "often accompanied by subtle disruptions in the environment",
    "Additional activity has been noted from", "though they do not appear hostile at present",
    "Field logs: instrumentation drift observed; recalibration recommended",
    "Night-vision readout noisy; expect degraded identification accuracy",
    "Keep monitoring the skies - aerial disturbances are not uncommon in this sector",
    "Communications are patchy; maintain line-of-sight whenever possible",
    "This sector has intermittent interference from legacy comm buoys",
    "Supply convoys have had to reroute due to unstable terrain conditions",
    "HQ analysts believe these disturbances are precursors to a larger conflict",
    "Maintain operational secrecy. HQ will expect a full after-action report.",
    "Use hand signals when verbal comms might reveal position",
    "Draw minimal bloodline; photographic evidence is priority",
    "Re-route patrols to avoid recurring sinkholes along Route 3",
    "Small salvageable components can be turned over to engineering",
    "Confirm target count by visual confirmation, not by sensor alone",
    "Use thermal masking as a decoy if pursuit is necessary",
    "Use nonlethal methods when the objective allows for capture",
    "Suppression fire authorized only on confirmed hostile contacts",
    "Remember that terrain is swampy near the east and rocky to the west",
    "Pre-brief: target identity confirmed via three sensors or witness corroboration",
    "Keep visual on the northern perimeter; movement correlated with wind bursts",
    "HQ requests photos with scale markers for each contact",
    "Maintain chain-of-custody for discovered artifacts",
    "Long-range sensors indicate sporadic bursts of radiation, possibly linked to latent evolutions",
    "If the target flees, pursue only after authorization from overwatch",
    "Avoid bright illumination near suspected nests - it agitates inhabitants",
    "Civilians in nearby settlements are growing anxious and require reassurance",
    "Multiple witness statements - low confidence - indicate movement at dawn",
    "Calm, low-frequency calls appear to pacify the group temporarily",
    "Radio checkpoint at 02:00 to confirm continued presence",
    "Field units have been reporting strange anomalies in energy readings across the valley",
    "Keep environmental samples for lab analysis (soil, residue, fur)",
    "Intercept logs indicate nonlocal movement patterns at dusk",
    "Tactical note: terrain is uneven; brace for elevation change",
    "Keep a secondary escape corridor clear at all times",
    "Small scuffle reported at the eastern fence line; no human injury",
    "Target behavior escalates when food sources are nearby",
    "Extract value intelligence before demolition where possible",
    "HQ reminder: do not deviate from the mission statement unless critical",
    "Thermal cameras logged irregular heat signatures near the treeline",
    "Local populations call them the 'yellow mouse' in nearby villages",
    "Reports from scouts mention hostile encounters but details remain unclear",
    "Mark LZ candidates; do not land within visible ashfall",
    "We are observing increased migration patterns from the northern ridge",
    "This region's flora contains irritants; PPE is advised for all personnel",
    "Tactical: maintain radio discipline; only encrypted channels unless urgent",
    "Refer to the terrain map appendix (sector colors may be outdated)",
    "Document each engagement with timestamped multimedia assets",
]

# ==============================================================================
# 2. DATA PROCESSING AND SYNTHETIC DATA GENERATION
# ==============================================================================
class PromptProcessor:
    """Processes and generates training data for the NLP model."""

    def __init__(self):
        self.filler_phrases = TACTICAL_FILLER

    def load_train_prompts(self, train_prompts_path):
        """Load the original training prompts"""
        with open(train_prompts_path, 'r', encoding='utf-8') as f:
            train_data = json.load(f)

        processed_data = []
        for item in train_data:
            # Extract target from simple prompts like "Kill: pikachu"
            prompt = item['prompt']
            target = prompt.split(": ")[1].strip().lower() if ": " in prompt else prompt.split(" ")[1].strip().lower()

            if target in CLASSES:
                processed_data.append({
                    "prompt": prompt,
                    "target": target,
                    "image_id": item.get("image_id", "")
                })

        return processed_data

    def generate_synthetic_prompts(self, num_prompts=10000):
        """Generate diverse synthetic prompts for robust training"""
        print(f"🧠 Generating {num_prompts} diverse tactical prompts...")
        prompts = []

        # Ensure balanced dataset
        prompts_per_class = num_prompts // len(CLASSES)

        for target in CLASSES:
            for _ in range(prompts_per_class):
                strategy = random.choice([
                    self._gen_tactical_report,
                    self._gen_long_tactical_with_distractors,
                    self._gen_buried_instruction,
                    self._gen_multiple_mentions_with_target,
                    self._gen_ambiguous_prompt,
                    self._gen_negated_prompt,
                    self._gen_instruction_emphasis,
                    self._gen_many_distractors_one_target,
                    self._gen_complex_scenario,
                ])
                prompts.append(strategy(target))

        # Fill remaining slots
        remaining = num_prompts - len(prompts)
        for _ in range(remaining):
            target = random.choice(CLASSES)
            strategy = random.choice([
                self._gen_tactical_report,
                self._gen_long_tactical_with_distractors,
                self._gen_complex_scenario,
            ])
            prompts.append(strategy(target))

        random.shuffle(prompts)
        print(f"✅ Generated {len(prompts)} tactical prompts.")
        return prompts

    def _get_synonyms(self, pokemon):
        return [k for k, v in POKEMON_SYNONYMS.items() if v == pokemon] + [pokemon]

    def _get_other_pokemon_mentions(self, target_pokemon):
        """Generate mentions of other pokemon as distractors"""
        others = [p for p in CLASSES if p != target_pokemon]
        mentions = []
        for other in others:
            synonyms = self._get_synonyms(other)
            mentions.extend(random.sample(synonyms, min(3, len(synonyms))))
        return mentions

    def _gen_tactical_report(self, target):
        """Generate tactical reports with diverse structures"""
        target_alias = random.choice(self._get_synonyms(target))
        distractors = self._get_other_pokemon_mentions(target)
        action = random.choice(ACTION_WORDS)

        # Build a realistic tactical report
        report_parts = [
            "HQ REPORT: Situation analysis regarding unusual activity of",
            random.choice(distractors), "in this operational zone."
        ]

        # Add multiple distractor mentions
        for _ in range(random.randint(2, 4)):
            distractor = random.choice(distractors)
            report_parts.extend([
                random.choice(self.filler_phrases),
                f"Scouts described sightings of {distractor} moving in small clusters, often accompanied by subtle disruptions in the environment."
            ])

        # Add the actual target instruction
        instruction_templates = [
            f"Order: {action} any {target_alias} encountered and maintain standoff until cleared.",
            f"Mission objective: identify and {action} the {target_alias} while avoiding non-targets.",
            f"Execute {action} of confirmed {target_alias} contacts; await extraction orders.",
            f"Your mission is to {action} all {target_alias} encountered in the area without hesitation."
        ]

        report_parts.extend([
            random.choice(self.filler_phrases),
            random.choice(instruction_templates),
            "Maintain operational secrecy. HQ will expect a full after-action report."
        ])

        return {"prompt": " ".join(report_parts), "target": target}

    def _gen_long_tactical_with_distractors(self, target):
        """Generate very long tactical reports with many distractors"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)

        # Start with header
        parts = ["HQ REPORT: Situation analysis regarding unusual activity in this operational zone."]

        # Add lots of tactical filler and distractors
        for _ in range(random.randint(6, 10)):
            parts.append(random.choice(self.filler_phrases))
            if random.random() < 0.7:  # 70% chance to add distractor
                distractor = random.choice(self._get_other_pokemon_mentions(target))
                parts.append(f"Scouts described sightings of {distractor} moving in small clusters, often accompanied by subtle disruptions in the environment.")

        # Bury the actual instruction deep in the text
        instruction = f"Priority: {action} the {target_alias} groups and prevent them from regrouping."
        insert_pos = len(parts) // 2  # Insert in middle
        parts.insert(insert_pos, instruction)

        # Add more filler after
        for _ in range(random.randint(2, 4)):
            parts.append(random.choice(self.filler_phrases))

        parts.append("Maintain operational secrecy. HQ will expect a full after-action report.")

        return {"prompt": " ".join(parts), "target": target}

    def _gen_buried_instruction(self, target):
        """Generate prompts where the real instruction is buried deep"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)

        # Lots of filler
        filler_text = " ".join([random.choice(self.filler_phrases) for _ in range(6)])

        # Buried instruction
        instruction = f"After all that analysis, the final order from high command is to {action} the {target_alias}."

        more_filler = " ".join([random.choice(self.filler_phrases) for _ in range(4)])

        full_prompt = f"{filler_text} {instruction} {more_filler}"

        return {"prompt": full_prompt, "target": target}

    def _gen_multiple_mentions_with_target(self, target):
        """Generate prompts with multiple pokemon mentions but clear target"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)
        distractors = self._get_other_pokemon_mentions(target)

        parts = ["HQ REPORT: Multiple contacts detected in the operational zone."]

        # Mention all pokemon as present but not hostile
        for distractor in distractors[:2]:
            parts.append(f"Additional activity has been noted from {distractor} groups nearby, though they do not appear hostile at present.")

        # Add filler
        parts.extend([random.choice(self.filler_phrases) for _ in range(2)])

        # Clear target instruction
        parts.append(f"Directives: {action} all {target_alias} in the AO and maintain perimeter security.")

        parts.append("Maintain operational secrecy. HQ will expect a full after-action report.")

        return {"prompt": " ".join(parts), "target": target}

    def _gen_ambiguous_prompt(self, target):
        """Generate prompts with ambiguous instructions to make learning harder"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)
        distractors = self._get_other_pokemon_mentions(target)

        parts = ["HQ REPORT: Multiple high-priority targets detected in the operational zone."]

        # Mention multiple pokemon with action words
        for distractor in distractors[:2]:
            distractor_action = random.choice(ACTION_WORDS)
            parts.append(f"Priority: {distractor_action} all {distractor} groups in sector 7.")

        # Add the actual target instruction
        parts.append(f"Secondary objective: {action} any {target_alias} encountered in the area.")

        # Add more ambiguity
        parts.append("Note: Target priorities may change based on real-time intelligence.")

        parts.append("Maintain operational secrecy. HQ will expect a full after-action report.")

        return {"prompt": " ".join(parts), "target": target}

    def _gen_negated_prompt(self, target):
        """Generate prompts with negated instructions to make learning harder"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)
        distractors = self._get_other_pokemon_mentions(target)

        parts = ["HQ REPORT: Complex tactical situation developing in the operational zone."]

        # Add negated instructions for distractors
        for distractor in distractors[:2]:
            negated_action = random.choice(NEGATION_WORDS)
            distractor_action = random.choice(ACTION_WORDS)
            parts.append(f"Important: {negated_action} {distractor_action} {distractor} units; they are not hostile.")

        # Add the actual target instruction
        parts.append(f"Primary directive: {action} all {target_alias} forces on sight.")

        parts.append("Maintain operational secrecy. HQ will expect a full after-action report.")

        return {"prompt": " ".join(parts), "target": target}

    def _gen_instruction_emphasis(self, target):
        """Generate prompts with clear instructions amidst many distractors"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)
        distractors = self._get_other_pokemon_mentions(target)

        parts = ["HQ REPORT: Multiple contacts detected in the operational zone."]

        # Add many distractor mentions
        for _ in range(random.randint(8, 12)):
            parts.append(random.choice(self.filler_phrases))
            distractor = random.choice(distractors)
            parts.append(f"Scouts described sightings of {distractor} moving in small clusters, often accompanied by subtle disruptions in the environment.")

        # Add a very clear instruction
        instruction_templates = [
            f"CRITICAL ORDER: {action.upper()} ALL {target_alias.upper()} UNITS IMMEDIATELY.",
            f"PRIMARY DIRECTIVE: {action} the {target_alias} above all other targets.",
            f"MISSION CRITICAL: Focus all efforts on {action}ing the {target_alias}."
        ]

        parts.append(random.choice(instruction_templates))

        # Add more distractors after the instruction
        for _ in range(random.randint(3, 5)):
            parts.append(random.choice(self.filler_phrases))
            distractor = random.choice(distractors)
            parts.append(f"Additional activity has been noted from {distractor} groups nearby.")

        parts.append("Maintain operational secrecy. HQ will expect a full after-action report.")

        return {"prompt": " ".join(parts), "target": target}

    def _gen_many_distractors_one_target(self, target):
        """Generate prompts with many distractors but only one target instruction"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)
        distractors = self._get_other_pokemon_mentions(target)

        parts = ["HQ REPORT: Complex multi-species activity detected across the operational zone."]

        # Add many distractor mentions
        for _ in range(random.randint(10, 15)):
            parts.append(random.choice(self.filler_phrases))
            distractor = random.choice(distractors)
            parts.append(f"Scouts described sightings of {distractor} moving in coordinated patterns.")

        # Add a single, clear instruction about the target
        parts.append(f"Despite multiple contacts, priority remains: {action} all {target_alias} encountered.")

        # Add more distractors
        for _ in range(random.randint(3, 5)):
            parts.append(random.choice(self.filler_phrases))
            distractor = random.choice(distractors)
            parts.append(f"Additional activity has been noted from {distractor} groups nearby.")

        parts.append("Maintain operational secrecy. HQ will expect a full after-action report.")

        return {"prompt": " ".join(parts), "target": target}

    def _gen_complex_scenario(self, target):
        """Generate complex tactical scenarios with multiple layers of instructions"""
        target_alias = random.choice(self._get_synonyms(target))
        action = random.choice(ACTION_WORDS)
        distractors = self._get_other_pokemon_mentions(target)

        parts = ["HQ REPORT: Multi-layered tactical scenario unfolding in the operational zone."]

        # Add complex scenario setup
        for _ in range(random.randint(8, 12)):
            parts.append(random.choice(self.filler_phrases))

            # Add distractor mentions with tactical context
            if random.random() < 0.6:
                distractor = random.choice(distractors)
                parts.append(f"Scouts described sightings of {distractor} moving in small clusters, often accompanied by subtle disruptions in the environment.")

            # Add target mentions with tactical context
            if random.random() < 0.4:
                target_synonym = random.choice(self._get_synonyms(target))
                parts.append(f"Scouts described sightings of {target_synonym} moving in small clusters, often accompanied by subtle disruptions in the environment.")

        # Add multiple instructions at different levels
        instruction_positions = [
            len(parts) // 3,  # Early
            len(parts) // 2,  # Middle
            2 * len(parts) // 3,  # Late
        ]

        for pos in instruction_positions:
            if random.random() < 0.7:  # 70% chance to add an instruction at each position
                instruction = f"Priority: {action} the {target_alias} groups and prevent them from regrouping."
                parts.insert(pos, instruction)

        # Add final parts
        parts.extend([
            "Maintain operational secrecy. HQ will expect a full after-action report."
        ])

        return {"prompt": " ".join(parts), "target": target}

# ==============================================================================
# 3. MODEL TRAINING AND INFERENCE
# ==============================================================================
def create_data_loader(encodings, labels, batch_size=4, shuffle=True):
    """Create a DataLoader from encodings and labels"""
    input_ids = torch.tensor(encodings['input_ids'])
    attention_mask = torch.tensor(encodings['attention_mask'])
    labels = torch.tensor(labels, dtype=torch.long)

    dataset = TensorDataset(input_ids, attention_mask, labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

def train_model_improved(model, train_loader, val_loader, num_epochs=2, learning_rate=2e-5, device='cuda'):
    """Improved training loop with better regularization to prevent overfitting"""
    model.to(device)

    # Use AdamW with higher weight decay for regularization
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.95)

    best_val_accuracy = 0
    best_model_state = None
    patience = 1  # Early stopping patience (shorter for 2 epochs)
    patience_counter = 0

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        train_correct = 0
        train_total = 0

        print(f"\n--- Epoch {epoch + 1}/{num_epochs} ---")

        for batch_idx, (input_ids, attention_mask, labels) in enumerate(train_loader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
            optimizer.step()

            total_train_loss += loss.item()
            predictions = torch.argmax(logits, dim=-1)
            train_correct += (predictions == labels).sum().item()
            train_total += labels.size(0)

            if batch_idx % 200 == 0:
                print(f"  Batch {batch_idx}: Loss = {loss.item():.4f}")

        train_accuracy = train_correct / train_total
        avg_train_loss = total_train_loss / len(train_loader)

        # Validation phase
        model.eval()
        total_val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for input_ids, attention_mask, labels in val_loader:
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

                total_val_loss += loss.item()
                predictions = torch.argmax(logits, dim=-1)
                val_correct += (predictions == labels).sum().item()
                val_total += labels.size(0)

        val_accuracy = val_correct / val_total
        avg_val_loss = total_val_loss / len(val_loader)

        print(f"Epoch {epoch + 1}:")
        print(f"  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}")
        print(f"  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

        # Early stopping check
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict().copy()
            patience_counter = 0
            print(f"  ✅ New best validation accuracy: {best_val_accuracy:.4f}")
        else:
            patience_counter += 1
            print(f"  ⏳ No improvement in validation accuracy for {patience_counter} epochs")

            if patience_counter >= patience:
                print(f"  🛑 Early stopping triggered after {epoch + 1} epochs")
                break

        scheduler.step()

    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"\n🏆 Best validation accuracy: {best_val_accuracy:.4f}")

    return model

def predict_targets(model, tokenizer, prompts, device='cuda'):
    """Predict targets for a list of prompts"""
    model.to(device)
    model.eval()

    predictions = []

    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)

        with torch.no_grad():
            logits = model(**inputs).logits
            predicted_class_id = logits.argmax().item()
            predicted_target = model.config.id2label[predicted_class_id]

        predictions.append(predicted_target)

    return predictions

# ==============================================================================
# 4. MAIN PIPELINE
# ==============================================================================
def main():
    """Main NLP pipeline for Pokemon target classification"""
    print("🚀 Pokemon Tactical Strike - NLP Pipeline")
    print("=" * 70)

    # Check device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"🔧 Using device: {device}")

    # Clear GPU cache
    if device == "cuda":
        torch.cuda.empty_cache()

    MODEL_PATH = "./pokemon_nlp_model"

    # Check if we already have a trained model
    if os.path.exists(MODEL_PATH) and os.path.exists(os.path.join(MODEL_PATH, "config.json")):
        print("✅ Found existing trained model. Skipping training...")

        # --- 3. Process Test Prompts ---
        print("\n[STEP 3/3] Processing Test Prompts...")

        try:
            # Load test prompts with encoding error handling
            try:
                with open("test_prompts.json", 'r', encoding='utf-8') as f:
                    test_data = json.load(f)
            except UnicodeDecodeError:
                # Try different encoding if UTF-8 fails
                print("⚠️  UTF-8 encoding failed, trying latin-1...")
                with open("test_prompts.json", 'r', encoding='latin-1') as f:
                    test_data = json.load(f)

            test_prompts = [item['prompt'] for item in test_data]
            test_image_ids = [item['image_id'] for item in test_data]

            print(f"✅ Loaded {len(test_prompts)} test prompts")

            # Load the trained model
            loaded_tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
            loaded_model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, trust_remote_code=True)

            # Predict targets
            print("🔮 Predicting targets for test prompts...")
            predicted_targets = predict_targets(loaded_model, loaded_tokenizer, test_prompts, device=device)

            # Create output in the required format
            output_data = {"images": []}
            for image_id, target in zip(test_image_ids, predicted_targets):
                # Remove file extension from image_id
                image_id_without_ext = image_id.split('.')[0]

                # Determine protected classes (all classes except the target)
                protected = [c for c in CLASSES if c != target]

                output_data["images"].append({
                    "id": image_id_without_ext,
                    "target": [target],
                    "protected": protected
                })

            # Save output
            with open("simplified_test_prompts.json", 'w') as f:
                json.dump(output_data, f, indent=2)

            print(f"✅ Simplified output saved to 'simplified_test_prompts.json'")
            print(f"📊 Sample predictions:")
            for i in range(min(5, len(output_data["images"]))):
                print(f"   {output_data['images'][i]['id']}: target={output_data['images'][i]['target']}, protected={output_data['images'][i]['protected']}")

        except Exception as e:
            print(f"❌ Error processing test prompts: {e}")
            # Try to provide more detailed error information
            import traceback
            traceback.print_exc()

    else:
        # --- 1. Data Preparation ---
        print("\n[STEP 1/3] Preparing Training Data...")

        processor = PromptProcessor()

        # Load original training data
        try:
            train_data = processor.load_train_prompts("train_prompts.json")
            print(f"✅ Loaded {len(train_data)} original training prompts")
        except:
            print("⚠️  Could not load train_prompts.json, using synthetic data only")
            train_data = []

        # Generate synthetic data
        synthetic_data = processor.generate_synthetic_prompts(num_prompts=10000)
        print(f"✅ Generated {len(synthetic_data)} synthetic training prompts")

        # Combine data
        all_data = train_data + synthetic_data
        random.shuffle(all_data)

        texts = [item['prompt'] for item in all_data]
        labels = [label2id[item['target']] for item in all_data]

        # Show class distribution
        label_counts = Counter(labels)
        print("📊 Class distribution:")
        for class_id, count in label_counts.items():
            print(f"   {id2label[class_id]}: {count} samples")

        train_texts, val_texts, train_labels, val_labels = train_test_split(
            texts, labels, test_size=0.2, random_state=42, stratify=labels
        )
        print(f"--> Data split into {len(train_texts)} training and {len(val_texts)} validation samples.")

        # --- 2. Model Training ---
        print("\n[STEP 2/3] Training NLP Model...")

        MODEL_CHECKPOINT = "chandar-lab/NeoBERT"

        try:
            tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, trust_remote_code=True)

            # Load model
            model = AutoModelForSequenceClassification.from_pretrained(
                MODEL_CHECKPOINT,
                num_labels=len(CLASSES),
                id2label=id2label,
                label2id=label2id,
                trust_remote_code=True
            )
            print(f"--> Model '{MODEL_CHECKPOINT}' loaded successfully.")
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            return

        # Tokenization
        print("🔤 Tokenizing data...")
        train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=1024)
        val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=1024)

        # Create data loaders
        train_loader = create_data_loader(train_encodings, train_labels, batch_size=4, shuffle=True)
        val_loader = create_data_loader(val_encodings, val_labels, batch_size=8, shuffle=False)

        print("🚀 Starting model training...")
        try:
            model = train_model_improved(model, train_loader, val_loader, num_epochs=2, learning_rate=2e-5, device=device)

            # Save model and tokenizer
            model.save_pretrained(MODEL_PATH)
            tokenizer.save_pretrained(MODEL_PATH)
            print(f"✅ Model training complete. Saved to '{MODEL_PATH}'.")
        except Exception as e:
            print(f"❌ Training failed: {e}")
            return

        # --- 3. Process Test Prompts ---
        print("\n[STEP 3/3] Processing Test Prompts...")

        try:
            # Load test prompts with encoding error handling
            try:
                with open("test_prompts.json", 'r', encoding='utf-8') as f:
                    test_data = json.load(f)
            except UnicodeDecodeError:
                # Try different encoding if UTF-8 fails
                print("⚠️  UTF-8 encoding failed, trying latin-1...")
                with open("test_prompts.json", 'r', encoding='latin-1') as f:
                    test_data = json.load(f)

            test_prompts = [item['prompt'] for item in test_data]
            test_image_ids = [item['image_id'] for item in test_data]

            print(f"✅ Loaded {len(test_prompts)} test prompts")

            # Load the trained model
            loaded_tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
            loaded_model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, trust_remote_code=True)

            # Predict targets
            print("🔮 Predicting targets for test prompts...")
            predicted_targets = predict_targets(loaded_model, loaded_tokenizer, test_prompts, device=device)

            # Create output in the required format
            output_data = {"images": []}
            for image_id, target in zip(test_image_ids, predicted_targets):
                # Remove file extension from image_id
                image_id_without_ext = image_id.split('.')[0]

                # Determine protected classes (all classes except the target)
                protected = [c for c in CLASSES if c != target]

                output_data["images"].append({
                    "id": image_id_without_ext,
                    "target": [target],
                    "protected": protected
                })

            # Save output
            with open("simplified_test_prompts.json", 'w') as f:
                json.dump(output_data, f, indent=2)

            print(f"✅ Simplified output saved to 'simplified_test_prompts.json'")
            print(f"📊 Sample predictions:")
            for i in range(min(5, len(output_data["images"]))):
                print(f"   {output_data['images'][i]['id']}: target={output_data['images'][i]['target']}, protected={output_data['images'][i]['protected']}")

        except Exception as e:
            print(f"❌ Error processing test prompts: {e}")
            # Try to provide more detailed error information
            import traceback
            traceback.print_exc()

if __name__ == "__main__":
    main()

🚀 Pokemon Tactical Strike - NLP Pipeline
🔧 Using device: cuda
✅ Found existing trained model. Skipping training...

[STEP 3/3] Processing Test Prompts...
⚠️  UTF-8 encoding failed, trying latin-1...
✅ Loaded 200 test prompts
🔮 Predicting targets for test prompts...
✅ Simplified output saved to 'simplified_test_prompts.json'
📊 Sample predictions:
   img_00000: target=['pikachu'], protected=['charizard', 'bulbasaur', 'mewtwo']
   img_00001: target=['charizard'], protected=['pikachu', 'bulbasaur', 'mewtwo']
   img_00002: target=['bulbasaur'], protected=['pikachu', 'charizard', 'mewtwo']
   img_00003: target=['charizard'], protected=['pikachu', 'bulbasaur', 'mewtwo']
   img_00004: target=['mewtwo'], protected=['pikachu', 'charizard', 'bulbasaur']


In [ ]:
import os
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'My Drive', 'myfolder')
drive.mount(MOUNTPOINT)

Mounted at /content/gdrive
